In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
#Hyperparameters for the training and loading

device = "cuda " if torch.cuda.is_available() else "cpu"
lr =2e-4
batch_size = 64
image_size = 64
channel_img = 1
z_dim = 100
num_epochs = 5


In [ ]:
import torch
import torch.nn as nn
from typing import Tuple

original_dim = 784
intermediate_dim = 256
latent_dim = 2

class Encoder(nn.Module):
    def __init__(self, in_features: int, hidden_features: int, latent_features: int):
        super().__init__()

        self.hidden = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.ReLU()
        )

        self.z_mean = nn.Linear(hidden_features, latent_features)
        self.z_log_var = nn.Linear(hidden_features, latent_features)

    def reparameterize(self, mu: torch.Tensor, log_var: torch.Tensor) -> torch.Tensor:
        std = torch.exp(0.5 * log_var)
        epsilon = torch.randn_like(std)
        return mu + epsilon * std

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        h = self.hidden(x)
        mu = self.z_mean(h)
        log_var = self.z_log_var(h)
        z = self.reparameterize(mu, log_var)

        return mu, log_var, z

encoder = Encoder(
    in_features=original_dim,
    hidden_features=intermediate_dim,
    latent_features=latent_dim
)

dummy_input = torch.randn(32, original_dim)

mean, log_variance, z_sample = encoder(dummy_input)

print("Mean shape:", mean.shape)
print("Log Var shape:", log_variance.shape)
print("Z Sample shape:", z_sample.shape)

In [ ]:
def sampling (mean , log_variance):
  epsilon = torch.randn((batch_size, latent_dim) , mean= 0. )
  return mean + torch.exp(log_variance/2) * epsilon

In [ ]:
class Decoder(nn.Module):
  def __init__(self, latent_features:int , hidden_features:int , out_features :int):
    super(). __init__()


    self.decode = nn.Sequential(
        nn.Linear(latent_features, hidden_features),
        nn.ReLU(),

        nn.Linear (hidden_features, out_features),
        nn.Sigmoid()
    )


  def forward (self , z : torch.tensor)-> torch.tensor :
    return self.decode(z)


decoder = Decoder(latent_features=latent_dim ,
                  hidden_features=intermediate_dim,
                  out_features=original_dim)
dummy_z = torch.randn(32 , latent_dim)

reconstructed_x = decoder(dummy_z)
print("Reconstructed X shape:", reconstructed_x.shape) # Expected: torch.Size([32, 784])


In [ ]:
class VAE(nn.Module):
  def __init__(self , input_features : int, intermediate_features:int, latent_features:int  ):
    super(). __init__()

    self.encoder = Encoder(original_dim , intermediate_dim , latent_dim)
    self.decoder = Decoder (latent_dim , intermediate_dim , original_dim)

  def forward (self , x: torch.tensor ) -> Tuple[torch.tensor , torch.tensor , torch.tensor ]:
     mu , log_variance , z = self.encoder(x)
     reconstruction = self.decoder(z)

     return reconstruction , mu , log_variance

vae_model = VAE(original_dim,intermediate_dim,latent_dim)

dummy_input = torch.randn (32 ,original_dim)
reconstructed_output, mean, log_variance = vae_model(dummy_input)
print("Final Reconstructed Output Shape:", reconstructed_output.shape)




In [ ]:
%pip install torch-summary

In [ ]:
from torchsummary import summary

# Create an instance of our VAE model and move it to the correct device (e.g., GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_summary = VAE(original_dim, intermediate_dim, latent_dim).to(device)

# Provide the model and the input size (without the batch dimension)
summary(model_summary, input_size=(original_dim,))

In [ ]:
loss_fn =  torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vae_model.parameters() , lr = 0.001)

In [ ]:
import torch.nn.functional as F

def vae_loss(x, x_decoded_mean, z_log_var, z_mean, original_dim=original_dim):

    reconstruction_loss = F.binary_cross_entropy(x_decoded_mean, x, reduction='sum')


    kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - torch.exp(z_log_var))

    return reconstruction_loss + kl_loss

In [ ]:
%pip install tqdm

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F

NUM_EPOCHS: int = 10
BATCH_SIZE: int = 128
LEARNING_RATE: float = 0.001
original_dim = 784
intermediate_dim = 256
latent_dim = 2


transform_pipeline =  transforms.Compose ([
    transforms.ToTensor(),
    transforms.Lambda(lambda x : torch.flatten(x))

])

train_dataset = datasets.MNIST(root = "dataset", train=True , transform=transform_pipeline , download =True)
test_dataset= datasets.MNIST(root = "dataset", train=False , transform=transform_pipeline , download =True)


train_dataloader = DataLoader(dataset = train_dataset , batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)



images , labels = next(iter(train_dataloader))

device = "cuda:0" if torch.cuda.is_available () else "cpu"
vae_model = VAE(original_dim , intermediate_dim , latent_dim ). to(device)
optimizer = torch.optim.Adam(vae_model.parameters(), lr=LEARNING_RATE)


for epoch in range(NUM_EPOCHS):

  vae_model.train()
  train_loss = 0

  for batch_idx, (images, _) in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
    images= images.to(device)




    optimizer.zero_grad()
    reconstruction , mu , log_variance = vae_model(images)


    loss = vae_loss(images , reconstruction , mu , log_variance)

    loss.backward()

    optimizer.step()

    train_loss +=loss.item()




  avg_loss = train_loss / len(train_dataloader.dataset)
  print(f"====> Epoch: {epoch+1} Average loss: {avg_loss:.4f}")

In [ ]:
%pip install matplotlib

In [ ]:
import torch
import matplotlib.pyplot as plt
import torchvision.utils as vutils


device = "cuda:0" if torch.cuda.is_available () else "cpu"
vae_model = VAE(original_dim , intermediate_dim , latent_dim ). to(device)


vae_model.eval()

num_images_to_generate = 64
latent_dim = 2

with torch.no_grad():
    z_sample = torch.randn(num_images_to_generate, latent_dim).to(device)
    generated_images = vae_model.decoder(z_sample)

generated_images = generated_images.view(num_images_to_generate, 1, 28, 28)

img_grid = vutils.make_grid(generated_images.cpu(), nrow=8, padding=2, normalize=True)

plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("New Digits Generated by VAE")
plt.imshow(img_grid.permute(1, 2, 0))
plt.show()

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import torch.nn as nn
import torch
import torch.functional as F
from torch.utils.data import DataLoader , Dataset


Z_DIM =100
IMG_SHAPE =(1,28,28)



class Generator(nn.Module):
  def __init__(self , img_shape, Z_Dim):
   super(). __init__()

   self.img_shape = img_shape

   self.model =nn.Sequential(
    nn.Linear(Z_Dim , 128 ),
    nn.LeakyReLU(0.01),
    nn.Linear(128 ,28*28*1),
    nn.Tanh()

   )

  def forward (self , z):
    img_flat  = self.model (z)
    img = img_flat.view(img_flat.size(0) , *self.img_shape)
    return img




generator = Generator(img_shape =IMG_SHAPE , Z_Dim =Z_DIM ).to(device)
dummy_noise = torch.randn (64 ,Z_DIM).to(device)
generated_img = generator(dummy_noise)

print(f"Shape of generated images : {generated_img.shape}")

In [ ]:
import numpy as np

class Discriminator(nn.Module):

  def __init__(self , img_shape):
    super(). __init__()

    flat_features = int(np.prod(img_shape))

    self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(flat_features , 128),
        nn.LeakyReLU(0.01),
        nn.Linear(128 , 1),
        nn.Sigmoid()

    )

  def forward(self , img):
    return self.model(img)


IMG_SHAPE = (1,28,28)
discriminator = Discriminator(img_shape=IMG_SHAPE).to(device)

dummy_img = torch.randn(64 , *IMG_SHAPE).to(device)
prediction = discriminator(dummy_img)
print("Shape of prediction:", prediction.shape)

In [ ]:
import torch.optim as optim

class Build_GAN(nn.Module):
  def __init__(self , Generator ,  Discriminator):
    super(). __init__()

    LEARNING_RATE =0.002

    device = "cuda :0" if torch.cuda.is_available() else "cpu"
    discriminator = Discriminator(img_shape=IMG_SHAPE).to(device)
    generator = Generator(img_shape =IMG_SHAPE , Z_Dim =Z_DIM ).to(device)

    criterion = nn.BCELoss()
    opt_disc = optim.Adam(discriminator.parameters(), lr = LEARNING_RATE)
    opt_gen = optim.Adam(generator.parameters(), lr = LEARNING_RATE)




In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

BATCH_SIZE = 128
EPOCHS = 20

transform_pipeline = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
]
)


train_dataset = datasets.MNIST(root = "dataset", train=True , transform=transform_pipeline , download =True)
test_dataset= datasets.MNIST(root = "dataset", train=False , transform=transform_pipeline , download =True)


train_dataloader = DataLoader(dataset = train_dataset , batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch.optim as optim
import torch.nn as nn

LEARNING_RATE =0.002
criterion = nn.BCELoss()
opt_disc = optim.Adam(discriminator.parameters(), lr = LEARNING_RATE)
opt_gen = optim.Adam(generator.parameters(), lr = LEARNING_RATE)


for epochs in range(EPOCHS):

  for batch_idx , (real_imgs , _) in enumerate(train_dataloader):
    real_imgs = real_imgs . to(device)
    batch_size = real_imgs.size(0)

    real_labels = torch.ones (batch_size, 1).to (device)
    fake_labels = torch.zeros(batch_size , 1). to (device)
#DISCRIMINATOR'S TRAINING.
#START BY GENERATING FAKE IMAGES
    noise = torch.randn(batch_size , Z_DIM).to(device)
    fake_images = generator(noise)


    #Training on real images

    disc_real_pred =discriminator(real_imgs)
    loss_disc_real = criterion(disc_real_pred , real_labels)

    #Training on fake images

    disc_fake_pred= discriminator(fake_images.detach())
    loss_disc_fake = criterion (disc_fake_pred , fake_labels)


    loss_disc = (loss_disc_fake + loss_disc_real) / 2
    discriminator.zero_grad()
    loss_disc.backward()
    opt_disc.step()


    #Train the generator

    output = discriminator (fake_images)
    loss_gen = criterion(output , real_labels)

    generator.zero_grad()
    loss_gen.backward()
    opt_gen.step()


    if batch_idx % 200 == 0 :
      print(
          f"Epoch [{epochs} / {EPOCHS} Batch {batch_idx} / {len(train_dataloader)}]"
          f"Loss D : {loss_disc.item() : .4f} , Loss G : {loss_gen.item(): .4f}"
      )
      sample_images(generator, epoch, Z_DIM, device)

In [ ]:
import torch
import matplotlib.pyplot as plt
import torchvision.utils as vutils
from pathlib import Path

# Create a directory to save the generated images
Path("generated_images").mkdir(exist_ok=True)

def sample_images(generator, epoch: int, z_dim: int, device: str):
    """
    Generates and saves a grid of images from the generator.

    Args:
        generator (nn.Module): The trained generator model.
        epoch (int): The current epoch number (used for the filename).
        z_dim (int): The dimension of the latent space.
        device (str): The device to run the model on ('cuda' or 'cpu').
    """

    generator.eval()


    num_images = 25


    noise = torch.randn(num_images, z_dim).to(device)

    # Disable gradient calculation
    with torch.no_grad():

        fake_images = generator(noise)




    # normalize=True scales the [-1, 1] output to [0, 1] for display.
    img_grid = vutils.make_grid(fake_images.cpu(), nrow=5, normalize=True)


    plt.figure(figsize=(6, 6))
    plt.axis("off")
    plt.title(f"Generated Images at Epoch {epoch}")
    plt.imshow(img_grid.permute(1, 2, 0))


    plt.savefig(f"generated_images/epoch_{epoch}.png")
    plt.show()

    # Set the generator back to training mode
    generator.train()

Implemeneting Deep GANs

In [ ]:
import torch.nn as nn

class deep_gen(nn.Module):
  def __init__(self , Z_dim):
    super(). __init__()

    self.initial_block = nn.Sequential(
        nn.Linear(Z_dim , 256 *7 *7),


    )

    self. upsample_block = nn.Sequential (
        nn.ConvTranspose2d(256 , 128 , kernel_size =3, stride =2 ,  padding =1 , output_padding = 1),
        nn.BatchNorm2d (128),
        nn.LeakyReLU(0.01),


        nn.ConvTranspose2d (128 , 64 ,kernel_size =3 , stride =1 , padding =1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(0.01),



        nn.ConvTranspose2d(64 , 1 , kernel_size =3, stride =2 ,  padding =1 , output_padding = 1),
        nn.Tanh()

    )

  def forward (self , Z):
    x = self.initial_block(Z)

    x= x.view(x.size(0), 256 ,7 , 7)
    img = self. upsample_block(x)

    return img

In [ ]:
class deep_disc(nn.Module):
  def __init__(self , img_Shape):
    super().__init__()

    channels , _ , _ = img_Shape


    self.downsample_block = nn.Sequential(
       nn.Conv2d(channels  , 32 , kernel_size= 3 , stride = 2 , padding =1),
       nn.LeakyReLU(0.01),


       nn.Conv2d(32 , 64 , kernel_size= 3 , stride =2 , padding =1),
       nn.BatchNorm2d(64),
       nn.LeakyReLU(0.01),

      nn.Conv2d(64 , 128 , kernel_size = 3 , stride =2 , padding =1),
       nn.BatchNorm2d(128),
       nn.LeakyReLU(0.01),


       nn.Flatten(),


       nn.Linear(128 * 4 * 4 , 1),
       nn.Sigmoid()

    )


  def forward(self , img):
    return self.downsample_block(img)

In [ ]:
import torch.optim as optim

class Build_DCGAN(nn.Module):
  def __init__(self , deep_gen ,  deep_disc):
    super(). __init__()

    LEARNING_RATE =0.002

    device = "cuda :0" if torch.cuda.is_available() else "cpu"
    deep_discr = deep_disc(img_shape=IMG_SHAPE).to(device)
    deep_genr = deep_gen(img_Shape =IMG_SHAPE , Z_dim =Z_DIM ).to(device)

    criterion = nn.BCELoss()
    opt_disc = optim.Adam(deep_disc.parameters(), lr = LEARNING_RATE)
    opt_gen = optim.Adam(deep_gen.parameters(), lr = LEARNING_RATE)

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

def train_gan(epochs, batch_size, sample_interval, z_dim, device):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transform, download=True)
    data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    generator = deep_gen(z_dim).to(device)
    discriminator = deep_disc(img_Shape=(1, 28, 28)).to(device)

    criterion = nn.BCELoss()
    opt_gen = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    opt_disc = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    losses = []

    print("🚀 Starting Training...")
    for epoch in range(epochs):
        for batch_idx, (real_imgs, _) in enumerate(data_loader):
            real_imgs = real_imgs.to(device)
            current_batch_size = real_imgs.size(0)

            real_labels = torch.ones(current_batch_size, 1).to(device)
            fake_labels = torch.zeros(current_batch_size, 1).to(device)

            discriminator.zero_grad()

            pred_real = discriminator(real_imgs)
            loss_d_real = criterion(pred_real, real_labels)

            noise = torch.randn(current_batch_size, z_dim).to(device)
            fake_imgs = generator(noise)
            pred_fake = discriminator(fake_imgs.detach())
            loss_d_fake = criterion(pred_fake, fake_labels)

            d_loss = (loss_d_real + loss_d_fake) / 2
            d_loss.backward()
            opt_disc.step()

            generator.zero_grad()

            output = discriminator(fake_imgs)
            g_loss = criterion(output, real_labels)

            g_loss.backward()
            opt_gen.step()

        print(
            f"Epoch [{epoch+1}/{epochs}] | "
            f"D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}"
        )

        losses.append((d_loss.item(), g_loss.item()))

        if (epoch + 1) % sample_interval == 0:
            sample_images(generator, epoch + 1, z_dim, device)

    print("✅ Training Complete!")
    return losses

if __name__ == '__main__':
    EPOCHS = 50
    BATCH_SIZE = 128
    SAMPLE_INTERVAL = 5
    Z_DIM = 100
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    training_losses = train_gan(EPOCHS, BATCH_SIZE, SAMPLE_INTERVAL, Z_DIM, DEVICE)